In [ ]:
import folium
from folium.plugins import HeatMap
import numpy as np
import pandas as pd
import os
current_directory = os.getcwd()

In [ ]:
location=[32.7003, -90.6783]
weather_data = pd.read_csv('/Users/abdullaahrobins/Documents/GitHub/CS506Spring2021Repository/NaturalDisasterProject/Datasets/Tornado_Data_2010-2020.csv')
# folium.Map(location=location, zoom_start=3)
# heat_df = weather_data[['BEGIN_LAT', 'BEGIN_LON']]
# heat_df = heat_df.dropna(axis=0, subset=['BEGIN_LAT','BEGIN_LON'])
# HeatMap(data=weather_data.groupby(['BEGIN_LAT', 'BEGIN_LON']).mean().reset_index().values.tolist(), radius=8, max_zoom=4).add_to(base_map)
# base_map.save('tornado_heat_map.html')

In [ ]:
# info = weather_data.info()
# state_occ = weather_data['STATE_ABBR'].value_counts()
# df = weather_data['STATE_ABBR'].value_counts().rename_axis('states').reset_index(name='counts')

In [ ]:
# url = (
#     "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
# )
# state_geo = f"{url}/us-states.json"

# m = folium.Map(location=location, zoom_start=3)

# folium.Choropleth(
#     geo_data=state_geo,
#     name="choropleth",
#     data=df,
#     columns=["states", "counts"],
#     key_on="feature.id",
#     fill_color="YlGn",
#     fill_opacity=0.7,
#     line_opacity=0.2,
#     legend_name="Tornado Occurrances",
# ).add_to(m)

# folium.LayerControl().add_to(m)

In [ ]:
# heat_data = [[row['BEGIN_LAT'],row['BEGIN_LON']] for index, row in heat_df.iterrows()]
# HeatMap(heat_data).add_to(base_map)

In [ ]:
# base_map.save('heatmap_weather.html')
# base_map.save('choropleth_weather.html')

In [ ]:
weather_data['BEGIN_DATE'] = pd.to_datetime(weather_data['BEGIN_DATE'])

In [ ]:
import requests
from datetime import timedelta
import json
'''
Formatting:
'start': '2010-01-01',
'end': '2018-12-31',
'''
c = 0
frames = []
for index, row in weather_data.iterrows():
    c+=1
    if c>50:
        break
    end_date = row.BEGIN_DATE
    begin_date = end_date - timedelta(weeks=8)
    lat = row.BEGIN_LAT
    lon = row.BEGIN_LON
    r = requests.get('https://api.oikolab.com/weather',
                     params={'param': ['temperature', 'wind_speed', 'surface_solar_radiation', 'relative_humidity', 'surface_pressure', 'total_precipitation'],
                             'start': begin_date,
                             'end': end_date,
                             'freq': 'D',
                             'lat': lat,
                             'lon': lon,
                             'api-key': 'b03998e660494c2e8165d41369c13248'}
                     )
    w_data = json.loads(r.json()['data'])
    history_df = pd.DataFrame(index=pd.to_datetime(w_data['index'], 
                                       unit='s'),
                  data=w_data['data'],
                  columns=w_data['columns'])
    history_df['latitude'] = lat
    history_df['longitude'] = lon
    frames.append(history_df)
result = pd.concat(frames)
result.index.name = 'datetime'
result.to_csv("historical_data.csv", encoding='utf-8', index=False)

In [ ]:
import requests
from datetime import timedelta
import json
from itertools import islice

'''
Formatting:
'start': '2010-01-01',
'end': '2018-12-31',
'''
c = 0
frames = []
for index, row in islice(weather_data.iterrows(),51,None):
    end_date = row.BEGIN_DATE
    begin_date = end_date - timedelta(weeks=8)
    lat = row.BEGIN_LAT
    lon = row.BEGIN_LON
    r = requests.get('https://api.oikolab.com/weather',
                     params={'param': ['temperature', 'wind_speed', 'surface_solar_radiation', 'relative_humidity', 'surface_pressure', 'total_precipitation'],
                             'start': begin_date,
                             'end': end_date,
                             'freq': 'D',
                             'lat': lat,
                             'lon': lon,
                             'api-key': 'b03998e660494c2e8165d41369c13248'}
                     )
    w_data = json.loads(r.json()['data'])
    history_df = pd.DataFrame(index=pd.to_datetime(w_data['index'], 
                                       unit='s'),
                  data=w_data['data'],
                  columns=w_data['columns'])
    history_df['latitude'] = lat
    history_df['longitude'] = lon
    frames.append(history_df)
result2 = pd.concat(frames)
result2.index.name = 'datetime'
result2.to_csv("historical_data2.csv", encoding='utf-8')

In [ ]:
result_fin = pd.concat([result,result2])
result_fin.to_csv("historical_data_fin.csv", encoding='utf-8')
